# Denoising Diffusion Probabilistic Models

In [ ]:
import torch 
import torch.nn as nn

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))
    
class DownBlock(nn.Module):
    def __init__(self, filters, in_channels):
        super(DownBlock, self).__init__()
        conv_blocks = [ConvBlock(in_channels, filters[0])]
        for i in range(1, len(filters) - 1):
            conv_blocks.append(ConvBlock(filters[i-1], filters[i]))
        
        conv_blocks.append(ConvBlock(filters[-2], filters[-1]))
        self.conv_blocks = nn.Sequential(*conv_blocks)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        outputs = []
        for conv_block in self.conv_blocks:
            x = conv_block(x)
            outputs.append(x)
            x = self.maxpool(x)

        return outputs

class UNet(nn.Module):
    def __init__(self, filters, in_channels, out_channels):
        super(UNet, self).__init__()
        
